In [1]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


import numpy as np
from scipy.stats import linregress

import matplotlib.pyplot as plt
import hvplot.pandas

In [2]:
### import and read csv file

path = '../Output/full_indicators_by_country.csv'
full_indicators = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators.head(5)

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,0,Afghanistan,AFG,32.400000,91.555556,58.432353,2.681272,327.0,36.628111,0.000000,...,0.012778,0.270084,3.220058,10.011889,7.579333,9.741222,11.616111,20.0,3.294,218.749241
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,4.822222,5.576404,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.574444,0.162233,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289
3,3,American Samoa,ASM,0.000000,0.000000,14.150000,0.000000,0.0,11.422222,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000
4,4,Andorra,AND,100.000000,100.000000,39.983452,0.000000,0.0,6.466667,0.000000,...,10.430000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000


In [3]:
# Generate summary statistics
full_indicators.describe()

,index,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,...,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,101.000000,61.681253,84.540394,36.676581,1.040230,1037.241872,19.211691,22.630020,292.324973,-0.052812,...,4.753558,3.196350,9.233506,5.193298,6.648959,6.481303,6.710729,37.182266,1.696631,104.523066
std,58.745213,40.911296,26.359982,22.669960,1.502681,845.406017,10.010777,38.152011,1219.684353,0.985466,...,4.069545,5.106580,12.240198,4.865685,6.824429,5.512370,5.663596,22.798325,0.993109,313.069257
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.222222,0.000000,0.000000,-1.819292,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.500000,11.344444,82.050000,18.275213,0.206179,324.500000,10.493333,0.000000,113.581446,-0.822233,...,0.779444,0.000000,0.000000,2.169056,1.537556,2.705167,2.912944,23.000000,1.388500,3.422510
50%,101.000000,82.744444,99.744444,38.527215,0.583351,847.000000,16.022667,0.000000,124.388549,-0.232470,...,4.054444,1.192078,4.689504,4.011222,4.887333,5.386889,5.067667,36.000000,1.957000,23.178038
75%,151.500000,100.000000,100.000000,52.313291,1.279990,1613.000000,27.434056,40.770094,159.647858,0.577856,...,7.997222,4.522973,14.558174,6.933667,9.850444,9.135333,9.314222,52.500000,2.293000,65.736551
max,202.000000,100.000000,100.000000,81.080429,13.679517,3240.000000,45.817111,209.829585,12920.702312,2.245200,...,16.863333,35.286391,64.389653,30.662889,35.480000,30.466333,29.187111,90.000000,3.397000,3066.215540


In [4]:
# # Plot your data to see what's in your DataFrame
# full_indicators.hvplot.line(
#     width=800,
#     height=400,
#     rot=90
# )

---

### Prepare the Data

In [5]:
full_indicators.columns

Index(['index', 'country', 'iso3Code',
       'clean fuels for cooking (% of population)',
       'Access to electricity (% of population)', 'Agricultural land (% )',
       'Armed forces (% labor force)', 'precipitation (mm per year)',
       'Birth rate, crude (per 1,000 people)',
       'Central government debt, total (% of GDP)',
       'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
       'Cost of business start-up (% of GNI per capita)',
       'Current account balance (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Death rate, crude (per 1,000 people)',
       'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
       'doing business score', 'Fertility rate, total (births per woman)',
       '% Firms using banks', 'GDP growth (annual %)',
       'GDP per capita (constant 2015 US$)',
       'GDP per capita, PPP (constant 2021 international $)',
       'GNI per capita (2015 US$)', 'government  consump

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

indicators_scaled = StandardScaler().fit_transform(full_indicators[[

                                                                      'clean fuels for cooking (% of population)',
                                                                      'Access to electricity (% of population)', 'Agricultural land (% )',
                                                                      'Armed forces (% labor force)', 'precipitation (mm per year)',
                                                                      'Birth rate, crude (per 1,000 people)',
                                                                      'Central government debt, total (% of GDP)',
                                                                      'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
                                                                      'Cost of business start-up (% of GNI per capita)',
                                                                      'Current account balance (% of GDP)',
                                                                      'Current health expenditure (% of GDP)',
                                                                      'Death rate, crude (per 1,000 people)',
                                                                      'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
                                                                      'doing business score', 'Fertility rate, total (births per woman)',
                                                                      '% Firms using banks', 'GDP growth (annual %)',
                                                                      'GDP per capita (constant 2015 US$)',
                                                                      'GDP per capita, PPP (constant 2021 international $)',
                                                                      'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
                                                                      'Government expenditure on education, total (% of GDP)',
                                                                      'Human Capital Index',
                                                                      'Industry (including construction), value added (% of GDP)',
                                                                      'Inflation, consumer prices (annual %)', 'Level of water stress',
                                                                      'Life expectancy at birth, total (years)',
                                                                      'Military expenditure (% of GDP)', 'Population density',
                                                                      'Population growth (annual %)', '% of total population below 5 meters',
                                                                      'Population living in slums (% of urban population)',
                                                                      'Population, total', 'Poverty $2.15 a day (% of population)',
                                                                      'Poverty $3.65 a day (% of population)',
                                                                      'Poverty $6.85 a day (% of population)', '%  Poverty headcount ratio ',
                                                                      'Time to start a business (days)',
                                                                      'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                                                                      'Total debt service (% of GNI)',
                                                                      'Total debt service (% of exports of goods, services and primary income)',
                                                                      'Unemployment advanced education', 'Unemployment basic education ',
                                                                      'Unemployment intermediate education',
                                                                      'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                                                      'Corruption index 2023', 'Global_Peace_Index_2023', 'immigration_100k'
    

                                                                      
                                                                      ]])

In [7]:
# Create a DataFrame with the scaled data
indicators_scaled_df = pd.DataFrame(indicators_scaled, columns=[
                                                               
                                                               'clean fuels for cooking (% of population)',
                                                               'Access to electricity (% of population)', 'Agricultural land (% )',
                                                               'Armed forces (% labor force)', 'precipitation (mm per year)',
                                                               'Birth rate, crude (per 1,000 people)',
                                                               'Central government debt, total (% of GDP)',
                                                               'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
                                                               'Cost of business start-up (% of GNI per capita)',
                                                               'Current account balance (% of GDP)',
                                                               'Current health expenditure (% of GDP)',
                                                               'Death rate, crude (per 1,000 people)',
                                                               'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
                                                               'doing business score', 'Fertility rate, total (births per woman)',
                                                               '% Firms using banks', 'GDP growth (annual %)',
                                                               'GDP per capita (constant 2015 US$)',
                                                               'GDP per capita, PPP (constant 2021 international $)',
                                                               'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
                                                               'Government expenditure on education, total (% of GDP)',
                                                               'Human Capital Index',
                                                               'Industry (including construction), value added (% of GDP)',
                                                               'Inflation, consumer prices (annual %)', 'Level of water stress',
                                                               'Life expectancy at birth, total (years)',
                                                               'Military expenditure (% of GDP)', 'Population density',
                                                               'Population growth (annual %)', '% of total population below 5 meters',
                                                               'Population living in slums (% of urban population)',
                                                               'Population, total', 'Poverty $2.15 a day (% of population)',
                                                               'Poverty $3.65 a day (% of population)',
                                                               'Poverty $6.85 a day (% of population)', '%  Poverty headcount ratio ',
                                                               'Time to start a business (days)',
                                                               'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                                                               'Total debt service (% of GNI)',
                                                               'Total debt service (% of exports of goods, services and primary income)',
                                                               'Unemployment advanced education', 'Unemployment basic education ',
                                                               'Unemployment intermediate education',
                                                               'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                                               'Corruption index 2023', 'Global_Peace_Index_2023', 'immigration_100k'
                                                        
                                                               ])

# # Copy the crypto names from the original DataFrame
indicators_scaled_df['country'] = full_indicators['country']
# indicators_scaled_df['iso3Code'] = full_indicators['iso3Code']



# Display the scaled DataFrame
indicators_scaled_df.head()

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,country
0,-0.717495,0.266787,0.962047,1.094776,-0.842196,1.744068,-0.594620,-0.120339,-1.340081,-0.071295,...,-0.574455,-0.492501,0.992769,0.136667,0.592845,0.868266,-0.755527,1.612430,0.365761,Afghanistan
1,0.484541,0.587085,0.252303,-0.284268,0.530946,-0.861323,1.461151,-0.140730,-0.454941,-0.264877,...,0.467228,0.646547,1.641097,0.587503,1.528103,1.042535,-0.008014,0.113429,0.299289,Albania
2,0.931050,0.574831,-0.854456,1.059723,-1.124414,0.388226,-0.594620,-0.111938,-0.597012,-0.256378,...,-0.595627,-0.695907,1.774375,0.272071,0.413997,0.966463,-0.051986,0.417267,-0.033128,Algeria
3,-1.511410,-3.215078,-0.996132,-0.693961,-1.229949,-0.780032,-0.594620,-0.240265,1.489307,-0.579959,...,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691,American Samoa
4,0.938946,0.587930,0.146231,-0.693961,-1.229949,-1.276278,-0.594620,-0.240265,1.286279,-0.579959,...,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691,Andorra


In [8]:
## Rename the indicators to be more ... friendly

indicators_scaled_df = indicators_scaled_df.rename(columns={


                                            'Access to clean fuels and technologies for cooking (% of population)'    :   'clean fuels for cooking (% of population)'  ,
                                            'Access to electricity (% of population)'    :   'Access to electricity (% of population)'  ,
                                            'Agricultural land (% of land area)'    :   'Agricultural land (% )'  ,
                                            'Armed forces personnel (% of total labor force)'    :   'Armed forces (% labor force)'  ,
                                            'Average precipitation in depth (mm per year)'    :   'precipitation (mm per year)'  ,
                                            'Birth rate crude (per 1000 people)'    :   'Birth rate crude'  ,
                                            'Central government debt total (% of GDP)'    :   'government debt (% of GDP)'  ,
                                            'Consumer price index (2010 = 100) Control of Corruption: Estimate'    :   'Consumer price index (2010)'  ,
                                            'Cost of business start-up procedures (% of GNI per capita)'    :   'Cost of business start-up (% of GNI per capita)'  ,
                                            'Current account balance (% of GDP)'    :   'Current account balance (% of GDP)'  ,
                                            'Current health expenditure (% of GDP)'    :   'Current health expenditure (% of GDP)'  ,
                                            'Death rate crude (per 1000 people)'    :   'Death rate crude (per 1000 people)'  ,
                                            'Debt service (PPG and IMF only % of exports of goods services and primary income)'    :   'Debt service'  ,
                                            'Ease of doing business score (0 = lowest performance to 100 = best performance)'    :   'doing business score'  ,
                                            'Fertility rate total (births per woman)'    :   'Fertility rate per woman'  ,
                                            'Firms using banks to finance investment (% of firms)'    :   '% Firms using banks'  ,
                                            'GDP growth (annual %) GDP per capita (constant 2015 US$)'    :   'GDP per capita growth '  ,
                                            'GDP per capita PPP (constant 2021 international $)'    :   'GDP per capita PPP ( 2021)'  ,
                                            'GNI per capita (constant 2015 US$)'    :   'GNI per capita (2015 US$)'  ,
                                            'General government final consumption expenditure (% of GDP)'    :   'government  consumption exp (% of GDP)'  ,
                                            'Government expenditure on education total (% of GDP)'    :   'Government expenditure on education total (% of GDP)'  ,
                                            'Human Capital Index (HCI) (scale 0-1)'    :   'Human Capital Index'  ,
                                            'Industry (including construction) value added (% of GDP)'    :   'Industry (% of GDP)'  ,
                                            'Inflation consumer prices (annual %)'    :   'Inflation consumer prices (%)'  ,
                                            'Level of water stress: freshwater withdrawal as a proportion of available freshwater resources'    :   'Level of water stress'  ,
                                            'Life expectancy at birth total (years)'    :   'Life expectancy at birth total (years)'  ,
                                            'Military expenditure (% of GDP)'    :   'Military expenditure (% of GDP)'  ,
                                            'Population density (people per sq. km of land area)'    :   'Population density'  ,
                                            'Population growth (annual %)'    :   'Population growth (annual %)'  ,
                                            'Population living in areas where elevation is below 5 meters (% of total population)'    :   '% of total population below 5 meters'  ,
                                            'Population living in slums (% of urban population)'    :   'Population living in slums (% of urban population)'  ,
                                            'Population total'    :   'Population total'  ,
                                            'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)'    :   'Poverty $2.15 a day (% of population)'  ,
                                            'Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population)'    :   'Poverty $3.65 a day (% of population)'  ,
                                            'Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population)'    :   'Poverty $6.85 a day (% of population)'  ,
                                            'Poverty headcount ratio at national poverty lines (% of population)'    :   '%  Poverty headcount ratio '  ,
                                            'Time required to start a business (days)'    :   'Time to start a business (days)'  ,
                                            'Total alcohol consumption per capita (liters of pure alcohol projected estimates 15+ years of age)'    :   'Alcohol consumption per capita'  ,
                                            'Total debt service (% of GNI)'    :   'Total debt service (% of GNI)'  ,
                                            'Total debt service (% of exports of goods services and primary income)'    :   'Total debt service (% of exports)'  ,
                                            'Unemployment with advanced education (% of total labor force with advanced education)'    :   'Unemployment advanced education'  ,
                                            'Unemployment with basic education (% of total labor force with basic education)'    :   'Unemployment basic education '  ,
                                            'Unemployment with intermediate education (% of total labor force with intermediate education)'    :   'Unemployment intermediate education'  ,
                                            'Unemployment total (% of total labor force) (modeled ILO estimate)]'    :   'Unemployment total'  ,
                                            'CPI_score_2023' : 'Corruption index 2023'


                                                    })
indicators_scaled_df.head(5)

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,country
0,-0.717495,0.266787,0.962047,1.094776,-0.842196,1.744068,-0.594620,-0.120339,-1.340081,-0.071295,...,-0.574455,-0.492501,0.992769,0.136667,0.592845,0.868266,-0.755527,1.612430,0.365761,Afghanistan
1,0.484541,0.587085,0.252303,-0.284268,0.530946,-0.861323,1.461151,-0.140730,-0.454941,-0.264877,...,0.467228,0.646547,1.641097,0.587503,1.528103,1.042535,-0.008014,0.113429,0.299289,Albania
2,0.931050,0.574831,-0.854456,1.059723,-1.124414,0.388226,-0.594620,-0.111938,-0.597012,-0.256378,...,-0.595627,-0.695907,1.774375,0.272071,0.413997,0.966463,-0.051986,0.417267,-0.033128,Algeria
3,-1.511410,-3.215078,-0.996132,-0.693961,-1.229949,-0.780032,-0.594620,-0.240265,1.489307,-0.579959,...,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691,American Samoa
4,0.938946,0.587930,0.146231,-0.693961,-1.229949,-1.276278,-0.594620,-0.240265,1.286279,-0.579959,...,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691,Andorra


In [9]:
indicators_scaled_df.columns

Index(['clean fuels for cooking (% of population)',
       'Access to electricity (% of population)', 'Agricultural land (% )',
       'Armed forces (% labor force)', 'precipitation (mm per year)',
       'Birth rate, crude (per 1,000 people)',
       'Central government debt, total (% of GDP)',
       'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
       'Cost of business start-up (% of GNI per capita)',
       'Current account balance (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Death rate, crude (per 1,000 people)',
       'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
       'doing business score', 'Fertility rate, total (births per woman)',
       '% Firms using banks', 'GDP growth (annual %)',
       'GDP per capita (constant 2015 US$)',
       'GDP per capita, PPP (constant 2021 international $)',
       'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
       'Governmen

In [10]:
# Organize columns

indicators_scaled_df = indicators_scaled_df[[

                                                'country', 
                                                'clean fuels for cooking (% of population)',
                                                'Access to electricity (% of population)', 'Agricultural land (% )',
                                            'Armed forces (% labor force)', 'precipitation (mm per year)',
                                            'Birth rate, crude (per 1,000 people)',
                                            'Central government debt, total (% of GDP)',
                                            'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
                                            'Cost of business start-up (% of GNI per capita)',
                                            'Current account balance (% of GDP)',
                                            'Current health expenditure (% of GDP)',
                                            'Death rate, crude (per 1,000 people)',
                                            'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
                                            'doing business score', 'Fertility rate, total (births per woman)',
                                            '% Firms using banks', 'GDP growth (annual %)',
                                            'GDP per capita (constant 2015 US$)',
                                            'GDP per capita, PPP (constant 2021 international $)',
                                            'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
                                            'Government expenditure on education, total (% of GDP)',
                                            'Human Capital Index',
                                            'Industry (including construction), value added (% of GDP)',
                                            'Inflation, consumer prices (annual %)', 'Level of water stress',
                                            'Life expectancy at birth, total (years)',
                                            'Military expenditure (% of GDP)', 'Population density',
                                            'Population growth (annual %)', '% of total population below 5 meters',
                                            'Population living in slums (% of urban population)',
                                            'Population, total', 'Poverty $2.15 a day (% of population)',
                                            'Poverty $3.65 a day (% of population)',
                                            'Poverty $6.85 a day (% of population)', '%  Poverty headcount ratio ',
                                            'Time to start a business (days)',
                                            'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                                            'Total debt service (% of GNI)',
                                            'Total debt service (% of exports of goods, services and primary income)',
                                            'Unemployment advanced education', 'Unemployment basic education ',
                                            'Unemployment intermediate education',
                                            'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                            'Corruption index 2023', 'Global_Peace_Index_2023', 'immigration_100k'
                                                                
                                            ]]
indicators_scaled_df.head(5)

,country,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,Afghanistan,-0.717495,0.266787,0.962047,1.094776,-0.842196,1.744068,-0.594620,-0.120339,-1.340081,...,-1.167821,-0.574455,-0.492501,0.992769,0.136667,0.592845,0.868266,-0.755527,1.612430,0.365761
1,Albania,0.484541,0.587085,0.252303,-0.284268,0.530946,-0.861323,1.461151,-0.140730,-0.454941,...,0.016914,0.467228,0.646547,1.641097,0.587503,1.528103,1.042535,-0.008014,0.113429,0.299289
2,Algeria,0.931050,0.574831,-0.854456,1.059723,-1.124414,0.388226,-0.594620,-0.111938,-0.597012,...,-1.029463,-0.595627,-0.695907,1.774375,0.272071,0.413997,0.966463,-0.051986,0.417267,-0.033128
3,American Samoa,-1.511410,-3.215078,-0.996132,-0.693961,-1.229949,-0.780032,-0.594620,-0.240265,1.489307,...,-1.170969,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691
4,Andorra,0.938946,0.587930,0.146231,-0.693961,-1.229949,-1.276278,-0.594620,-0.240265,1.286279,...,1.398308,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691


In [11]:
indicators_scaled_df = indicators_scaled_df.set_index(['country'])
indicators_scaled_df.head(5)

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,-0.717495,0.266787,0.962047,1.094776,-0.842196,1.744068,-0.594620,-0.120339,-1.340081,-0.071295,...,-1.167821,-0.574455,-0.492501,0.992769,0.136667,0.592845,0.868266,-0.755527,1.612430,0.365761
Albania,0.484541,0.587085,0.252303,-0.284268,0.530946,-0.861323,1.461151,-0.140730,-0.454941,-0.264877,...,0.016914,0.467228,0.646547,1.641097,0.587503,1.528103,1.042535,-0.008014,0.113429,0.299289
Algeria,0.931050,0.574831,-0.854456,1.059723,-1.124414,0.388226,-0.594620,-0.111938,-0.597012,-0.256378,...,-1.029463,-0.595627,-0.695907,1.774375,0.272071,0.413997,0.966463,-0.051986,0.417267,-0.033128
American Samoa,-1.511410,-3.215078,-0.996132,-0.693961,-1.229949,-0.780032,-0.594620,-0.240265,1.489307,-0.579959,...,-1.170969,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691
Andorra,0.938946,0.587930,0.146231,-0.693961,-1.229949,-1.276278,-0.594620,-0.240265,1.286279,-0.579959,...,1.398308,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [12]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [13]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(indicators_scaled_df)
    inertia.append(k_model.inertia_)


c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Window

In [14]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)
df_elbow.head()

,k,inertia
0,1,9947.000000
1,2,8537.296289
2,3,7853.474127
3,4,7179.782830
4,5,6921.395787


In [15]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_original_data = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve without PCA", 
    xticks=k
)

elbow_original_data


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [16]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [17]:
# Fit the K-Means model using the scaled DataFrame
model.fit(indicators_scaled_df)

c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [18]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
kmeans_predictions = model.predict(indicators_scaled_df)

# Print the resulting array of cluster values.
print(kmeans_predictions)

[2 3 3 0 0 1 0 3 3 0 0 3 0 0 1 0 3 0 3 1 0 3 3 3 3 3 0 0 0 1 1 3 2 2 0 2 1
 0 0 3 2 1 2 3 1 0 0 0 0 0 0 3 0 3 3 3 3 2 2 0 1 0 0 0 0 2 1 3 0 1 3 0 3 1
 1 0 2 3 0 0 0 1 3 3 2 0 0 0 3 0 3 3 1 0 2 0 3 0 3 2 0 3 1 1 2 0 0 0 0 2 1
 0 0 1 0 0 1 3 3 0 0 3 3 3 1 2 0 3 0 0 0 3 1 1 3 0 0 0 1 0 0 2 3 3 3 0 0 0
 0 0 0 1 3 0 1 0 1 3 0 1 0 0 0 0 0 2 3 2 0 3 3 0 2 3 0 0 2 3 1 0 0 1 0 0 3
 3 2 0 0 1 3 0 0 0 0 3 0 2 0 3 2 1 2]


In [19]:
# Create a copy of the scaled DataFrame
indicators_prediction_df = indicators_scaled_df.copy()

In [20]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
indicators_prediction_df['countries_cluster_original_data'] = kmeans_predictions

# Display the copy of the scaled DataFrame
indicators_prediction_df.head()

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,-0.717495,0.266787,0.962047,1.094776,-0.842196,1.744068,-0.594620,-0.120339,-1.340081,-0.071295,...,-0.574455,-0.492501,0.992769,0.136667,0.592845,0.868266,-0.755527,1.612430,0.365761,2
Albania,0.484541,0.587085,0.252303,-0.284268,0.530946,-0.861323,1.461151,-0.140730,-0.454941,-0.264877,...,0.467228,0.646547,1.641097,0.587503,1.528103,1.042535,-0.008014,0.113429,0.299289,3
Algeria,0.931050,0.574831,-0.854456,1.059723,-1.124414,0.388226,-0.594620,-0.111938,-0.597012,-0.256378,...,-0.595627,-0.695907,1.774375,0.272071,0.413997,0.966463,-0.051986,0.417267,-0.033128,3
American Samoa,-1.511410,-3.215078,-0.996132,-0.693961,-1.229949,-0.780032,-0.594620,-0.240265,1.489307,-0.579959,...,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691,0
Andorra,0.938946,0.587930,0.146231,-0.693961,-1.229949,-1.276278,-0.594620,-0.240265,1.286279,-0.579959,...,-0.627475,-0.756224,-1.069970,-0.976697,-1.178681,-1.187818,-1.634953,-1.712627,-0.334691,0


In [21]:
# Create a scatter plot using hvPlot by setting
    # `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
    # Color the graph points with the labels found using K-Means and
    # add the crypto name in the `hover_cols` parameter to identify
    # the cryptocurrency represented by each data point.

clusters_original_data = indicators_prediction_df.hvplot.scatter(
    x="immigration_100k", 
    y="Global_Peace_Index_2023", 
    by="countries_cluster_original_data",
    title = "Clusters with original data",
    hover_cols = 'country'
)

clusters_original_data

:NdOverlay   [countries_cluster_original_data]
   :Scatter   [immigration_100k]   (Global_Peace_Index_2023,country)

---

### Optimize Clusters with Principal Component Analysis.

In [22]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=4)

In [23]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
indicators_pca = pca.fit_transform(indicators_scaled_df)

# View the scaled PCA data
indicators_pca[0:5]

array([[ 3.23200025, -0.8283388 ,  0.48018802,  0.33434859],
       [-0.89190045, -2.48232244,  2.23019704, -1.04857731],
       [-0.27982685, -0.47773574,  0.92085112,  2.24533521],
       [ 0.6293702 ,  6.76548966,  1.40147728, -2.73509082],
       [-2.0243087 ,  5.0460066 ,  0.21156073, -1.89265386]])

In [24]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.20445913, 0.10960015, 0.06726142, 0.05579026])

In [25]:
# Use 'sum' on the explained varaibles array to obtain the total explained variance

total_explained_variance = sum(pca.explained_variance_ratio_)
print(f'The total explained variance is : {total_explained_variance}')

The total explained variance is : 0.43711096396712923


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 43.7%

In [26]:
# Create a new DataFrame with the PCA data.
indicators_pca_df = pd.DataFrame(
    indicators_pca,
    columns=["PCA1", "PCA2", 'PCA3', 'PCA4']
)

# Copy the crypto names from the original scaled DataFrame
indicators_pca_df['country'] = full_indicators['country']

# # Set the coin_id column as index
indicators_pca_df = indicators_pca_df.set_index(['country'])

# Display the scaled PCA DataFrame
indicators_pca_df.head(5)

,PCA1,PCA2,PCA3,PCA4
country,,,,
Afghanistan,3.232000,-0.828339,0.480188,0.334349
Albania,-0.891900,-2.482322,2.230197,-1.048577
Algeria,-0.279827,-0.477736,0.920851,2.245335
American Samoa,0.629370,6.765490,1.401477,-2.735091
Andorra,-2.024309,5.046007,0.211561,-1.892654


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [27]:
# Create a list with the number of k-values from 1 to 11
    # we already have the list
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [28]:
# Create an empty list to store the inertia values

inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model_pca = KMeans(n_clusters=i, random_state=1)
    k_model_pca.fit(indicators_pca_df)
    inertia_pca.append(k_model_pca.inertia_)

c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Window

In [29]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow_pca = pd.DataFrame(elbow_data_pca)
df_elbow_pca.head()


,k,inertia
0,1,4347.942759
1,2,3010.065977
2,3,2302.609150
3,4,1959.614633
4,5,1379.045686


In [30]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.

elbow_with_pca = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve_with_PCA", 
    xticks=k
)

elbow_with_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 5


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** yes

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [31]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [32]:
# Fit the K-Means model using the PCA data
model.fit(indicators_pca_df)

c:\Users\nazim\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [33]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
kmeans_predictions_pca = model.predict(indicators_pca_df)

# Print the resulting array of cluster values.
print(kmeans_predictions_pca)

[1 3 3 2 2 1 2 3 3 0 0 3 2 0 1 0 3 0 3 1 2 3 3 3 3 3 2 0 3 1 1 3 2 1 0 1 1
 0 0 3 2 1 1 3 1 0 0 2 0 0 0 3 3 3 3 3 3 2 2 0 1 3 0 0 2 3 1 3 0 1 3 3 3 1
 1 3 2 1 0 0 0 1 3 3 3 0 0 0 3 0 3 3 1 2 2 0 3 0 3 3 0 3 1 1 2 2 0 0 2 1 1
 0 2 1 0 2 1 3 3 3 2 3 3 3 1 1 2 3 0 2 0 3 1 1 3 2 0 0 1 2 3 2 3 3 3 0 0 2
 0 0 0 1 2 2 1 0 1 3 0 1 0 2 0 0 2 1 3 1 0 3 3 3 1 2 0 0 2 1 1 0 2 1 2 0 3
 3 2 2 2 1 3 0 0 0 0 3 2 2 3 3 2 1 1]


In [34]:
# Create a copy of the scaled PCA DataFrame

indicators_pca_new_df = indicators_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
indicators_pca_new_df['countries_cluster_withPCA'] = kmeans_predictions_pca

# Display the copy of the scaled PCA DataFrame
indicators_pca_new_df.head(5)

,PCA1,PCA2,PCA3,PCA4,countries_cluster_withPCA
country,,,,,
Afghanistan,3.232000,-0.828339,0.480188,0.334349,1
Albania,-0.891900,-2.482322,2.230197,-1.048577,3
Algeria,-0.279827,-0.477736,0.920851,2.245335,3
American Samoa,0.629370,6.765490,1.401477,-2.735091,2
Andorra,-2.024309,5.046007,0.211561,-1.892654,2


In [35]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
clusters_with_PCA = indicators_pca_new_df.hvplot.scatter(
    x="PCA1", 
    y="PCA2", 
    by="countries_cluster_withPCA",
    title = "Clusters with PCA" ,
    hover_cols = 'country'
)

clusters_with_PCA

:NdOverlay   [countries_cluster_withPCA]
   :Scatter   [PCA1]   (PCA2,country)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [36]:
# Composite plot to contrast the Elbow curves

plots = (elbow_original_data + elbow_with_pca).cols(1)
plots

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [37]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!


plots = (clusters_original_data + clusters_with_PCA).cols(1)
plots

:Layout
   .NdOverlay.I  :NdOverlay   [countries_cluster_original_data]
      :Scatter   [immigration_100k]   (Global_Peace_Index_2023,country)
   .NdOverlay.II :NdOverlay   [countries_cluster_withPCA]
      :Scatter   [PCA1]   (PCA2,country)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 

  1)  regarding the number of clusters
  the fact that the optimal k was the same with and without use of PCA suggests the following : 
  - PCA did not significantly distort the structure of the data relevant to clustering..
  - The original dataset likely had a strong cluster structure and the clustering information was present in the original feature set.
  - the clusters in the original dataset are well separated, so the PCA did not change significantly the separability.

by another hand this consistancy is positive because 
  - it gives more confidence in the clustering results


  2) regarding the clustering assignements
  we observe that both methods (with and without PCA) give the same clusters (whatever is the # of th cluster)
  all the coins belong to the same cluster in both methods as shown in the dataframe below

  this reinforce the conclusions above

In [38]:
# to have the detailled content of the clusters, we will merge the 2 dataframes (with original data and with PCA)
# than we will compare the content of each cluster
# no need to make further queries, the data is limited and allows us to conclude

clusters = pd.merge(indicators_pca_new_df, indicators_prediction_df, on='country')
clusters = clusters[['countries_cluster_original_data', 'countries_cluster_withPCA']].sort_values(['countries_cluster_withPCA'])

# clusters.head(5)
clusters

,countries_cluster_original_data,countries_cluster_withPCA
country,,
Iceland,0,0
Latvia,0,0
Germany,0,0
Russian Federation,0,0
Romania,0,0
...,...,...
Morocco,3,3
Montenegro,3,3
Mongolia,3,3


In [39]:
clusters.to_csv('../Output_analysis/Clusters_analysis.csv')

In [ ]:
# this export is needed to create the table in DB

cluster_to_print = clusters.reset_index()
cluster_to_print = cluster_to_print[['country', 'countries_cluster_withPCA']].rename(columns={'countries_cluster_withPCA':'cluster'})
cluster_to_print = cluster_to_print.sort_values('country')
cluster_to_print.to_csv('../Output_analysis/Clusters_DB.csv', index=False)
cluster_to_print.head()

,country,cluster
60,Afghanistan,1
148,Albania,3
149,Algeria,3
125,American Samoa,2
116,Andorra,2


In [41]:
### import and read csv file

path = '../Output/full_indicators_by_country.csv'
full_indicators_countries_df = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators_countries_df.head(5)

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,0,Afghanistan,AFG,32.400000,91.555556,58.432353,2.681272,327.0,36.628111,0.000000,...,0.012778,0.270084,3.220058,10.011889,7.579333,9.741222,11.616111,20.0,3.294,218.749241
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,4.822222,5.576404,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.574444,0.162233,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289
3,3,American Samoa,ASM,0.000000,0.000000,14.150000,0.000000,0.0,11.422222,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000
4,4,Andorra,AND,100.000000,100.000000,39.983452,0.000000,0.0,6.466667,0.000000,...,10.430000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000


In [42]:
new_df = pd.merge(full_indicators_countries_df, clusters, on = 'country' )
new_df

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
0,0,Afghanistan,AFG,32.400000,91.555556,58.432353,2.681272,327.0,36.628111,0.000000,...,3.220058,10.011889,7.579333,9.741222,11.616111,20.0,3.294,218.749241,2,1
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096,3,3
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289,3,3
3,3,American Samoa,ASM,0.000000,0.000000,14.150000,0.000000,0.0,11.422222,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0,2
4,4,Andorra,AND,100.000000,100.000000,39.983452,0.000000,0.0,6.466667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,198,Viet Nam,VNM,91.644444,99.744444,39.126963,0.943333,1821.0,15.627444,0.000000,...,5.989108,3.482444,1.287222,2.345889,1.785333,41.0,1.802,43.300050,0,3
199,199,West Bank and Gaza,PSE,0.000000,99.977778,68.829501,0.000000,402.0,29.736444,0.000000,...,0.000000,30.662889,22.519000,24.193444,25.038000,0.0,2.872,128.246720,3,3
200,200,"Yemen, Rep.",YEM,52.588889,72.344444,44.401176,0.503899,167.0,31.745667,0.000000,...,15.049805,0.000000,0.000000,0.000000,17.822111,16.0,3.397,17.149918,2,2
201,201,Zambia,ZMB,11.211111,41.877778,32.038066,0.266775,1020.0,35.655000,65.282245,...,14.786883,6.638556,6.436556,9.354556,5.490778,37.0,1.948,3.345682,1,1


In [43]:
# to have the population of each cluster
x = new_df.groupby('countries_cluster_withPCA')['country'].count()
x

countries_cluster_withPCA
0    55
1    44
2    41
3    63
Name: country, dtype: int64

In [44]:
# this is to test the list that will be used in the fnction below

indicators_list = full_indicators_countries_df.columns.to_list()
indicators_list = indicators_list[2:len(indicators_list)-1]
indicators_list

['iso3Code',
 'clean fuels for cooking (% of population)',
 'Access to electricity (% of population)',
 'Agricultural land (% )',
 'Armed forces (% labor force)',
 'precipitation (mm per year)',
 'Birth rate, crude (per 1,000 people)',
 'Central government debt, total (% of GDP)',
 'Consumer price index (2010 = 100)',
 'Control of Corruption: Estimate',
 'Cost of business start-up (% of GNI per capita)',
 'Current account balance (% of GDP)',
 'Current health expenditure (% of GDP)',
 'Death rate, crude (per 1,000 people)',
 'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
 'doing business score',
 'Fertility rate, total (births per woman)',
 '% Firms using banks',
 'GDP growth (annual %)',
 'GDP per capita (constant 2015 US$)',
 'GDP per capita, PPP (constant 2021 international $)',
 'GNI per capita (2015 US$)',
 'government  consumption exp (% of GDP)',
 'Government expenditure on education, total (% of GDP)',
 'Human Capital Index',
 'Industry (

In [45]:
# function that compares all the indicators against ONE (immigration flow or immigration_100K) and generate a DF with al the regression parameters

# arguments are 
        ## the dataframe name
        ## number of index columns to be exculded

def select_indicator(dataframe,col_index):
    
    # create list of indicator from the Dataframe consolidated
    indicators_list = dataframe.columns.to_list()
    indicators_list = indicators_list[col_index:len(indicators_list)-1]
    indicators_list

    # create list of returned values
    indic_selection = []

    # loop through the indicators
    for ind in indicators_list :
        col1 = 'immigration_100k'
        col2 = ind


        df_regression = dataframe[[col1,col2]]
        
        #drop empty values

        df_regression = df_regression.dropna(how= 'any')


        # identify outliers
        var = df_regression[col2]
        q1 = np.quantile(var, 0.25) 
        q3 = np.quantile(var, 0.75)   
        iqr = q3 - q1
        outliers = var[(var > (q3 + (iqr * 1.5))) | (var < (q1 - (iqr * 1.5)))].tolist()

        # Exclude  outliers
        x = ~df_regression[col2].isin(outliers)
        dataframe_filtered = df_regression[x]

        x_axis = dataframe_filtered[col1]
        y_axis = dataframe_filtered[col2]

        ## calculate slope and intercept of the regression equation

        (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
        

        # append regresson values in list of dictionaries
        indic_selection.append({
                            'indicator A' : col1,
                            'indicator B' : col2,
                            'slope' : slope,
                            'intercept' :  intercept, 
                            'rvalue' : rvalue, 
                            'pvalue' : pvalue, 
                            'std' : stderr,
                            "rvalue_abs" : abs(rvalue)

                            })
    
    # create DF from list of indicators compared
    selection_df = pd.DataFrame(indic_selection)
    selection_df = selection_df.sort_values('rvalue_abs', ascending=False)

    selection_df = selection_df.reset_index()    
    
    return selection_df.head(50)


In [46]:
# create empty list to store the results
results_list = []

# loop for each cluster and generate a linear regression DF
for i in range(0,4) :
    
    result_dict = {}
    
    cluster_num = i

    # filter the DF regarding the cluster

    clustered_df = new_df[new_df['countries_cluster_withPCA']==cluster_num]
    
    # prepare the DF to the linear regression
    clustered_df = clustered_df.drop(columns=['countries_cluster_original_data', 'countries_cluster_withPCA'])
    
    # run the linear regression
    Linear_regression_with_PCA = select_indicator(clustered_df,3)

    # export the reslt of the linear regression to EXCEL file
    file_destination = f'../Output_analysis/Linear_regression_with_PCA_cluster_number_{cluster_num}.xlsx'
    Linear_regression_with_PCA.to_excel(file_destination)

    # extract the max rvalue
    rvalue_max = Linear_regression_with_PCA['rvalue_abs'].max()

    # store the max rvalue in a dict to be appended in the list results
    result_dict = {
            'cluster' : i ,
            'rvalue' : rvalue_max
                }
    results_list.append(result_dict)

# create DF with list of results
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('rvalue', ascending=False)
results_df

,cluster,rvalue
3,3,0.438036
1,1,0.401879
2,2,0.377876
0,0,0.366653


In [47]:
# filter the DF regarding the cluster

## choose the cluster
cluster_num = 3

## filter
clustered_df = new_df[new_df['countries_cluster_withPCA']==cluster_num]
clustered_df = clustered_df.drop(columns=['countries_cluster_original_data', 'countries_cluster_withPCA'])
clustered_df.head()

,index,country,iso3Code,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
1,1,Albania,ALB,81.455556,99.977778,42.382186,0.614119,1485.0,10.610444,78.238375,...,4.822222,5.576404,17.127848,13.158667,10.648444,14.884000,12.600667,37.0,1.809,197.990096
2,2,Algeria,DZA,99.677778,99.655556,17.353871,2.628728,89.0,23.088556,0.000000,...,0.574444,0.162233,0.736472,13.805556,8.501111,8.757778,12.170889,36.0,2.110,94.177289
7,7,Argentina,ARG,99.844444,99.955556,43.163578,0.530716,591.0,15.106778,0.000000,...,8.198889,6.645575,41.525570,2.702000,8.230889,7.775444,8.468333,37.0,1.855,7.117405
8,8,Armenia,ARM,97.800000,99.933333,58.698513,3.256943,562.0,12.983778,29.623738,...,4.643333,13.312760,30.398428,11.418333,10.545778,12.555222,16.791111,47.0,2.052,69.704423
11,11,Azerbaijan,AZE,97.922222,100.000000,57.805163,1.637356,447.0,13.777778,8.073031,...,2.043333,4.232180,8.248160,5.175556,11.237778,4.988889,5.491778,23.0,2.248,25.319819


In [48]:
# run the function linear regression and compare all indicators by year

Linear_regression_with_PCA = select_indicator(clustered_df,3)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_with_PCA

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,8,immigration_100k,Control of Corruption: Estimate,6.933416e-04,-5.606968e-01,0.438036,0.000584,0.000190,0.438036
1,46,immigration_100k,Corruption index 2023,8.747384e-03,3.663190e+01,0.401313,0.001632,0.002644,0.401313
2,10,immigration_100k,Current account balance (% of GDP),-3.206583e-03,-3.384274e+00,-0.348814,0.007832,0.001162,0.348814
3,24,immigration_100k,"Industry (including construction), value added...",-6.415302e-03,2.668565e+01,-0.340085,0.006842,0.002290,0.340085
4,12,immigration_100k,"Death rate, crude (per 1,000 people)",1.285902e-03,6.781707e+00,0.318885,0.016602,0.000520,0.318885
5,3,immigration_100k,Armed forces (% labor force),-5.619850e-04,1.201524e+00,-0.307800,0.019848,0.000234,0.307800
6,16,immigration_100k,% Firms using banks,-8.234556e-03,1.508379e+01,-0.292025,0.020215,0.003453,0.292025
7,28,immigration_100k,Military expenditure (% of GDP),-5.875465e-04,1.376096e+00,-0.280765,0.032774,0.000268,0.280765
8,30,immigration_100k,Population growth (annual %),-5.648799e-04,8.548772e-01,-0.258885,0.042178,0.000272,0.258885
9,35,immigration_100k,Poverty $3.65 a day (% of population),-2.701308e-03,4.985088e+00,-0.252416,0.055930,0.001384,0.252416
